# <center>Композиции алгоритмов

Будем решать задачу кредитного скоринга.

#### Данные по кредитному скорингу представлены следующим образом:

##### Прогнозируемая  переменная
* SeriousDlqin2yrs	– наличие длительных просрочек выплат платежей за 2 года.

##### Независимые признаки
* age – возраст заёмщика (число полных лет);
* NumberOfTime30-59DaysPastDueNotWorse	– количество раз, когда заёмщик имел просрочку выплаты других кредитов 30-59 дней в течение последних двух лет;
* NumberOfTime60-89DaysPastDueNotWorse – количество раз, когда заёмщик имел просрочку выплаты других кредитов 60-89 дней в течение последних двух лет;
* NumberOfTimes90DaysLate – количество раз, когда заёмщик имел просрочку выплаты других кредитов более 90 дней;
* DebtRatio – ежемесячные отчисления на задолжености (кредиты, алименты и т.д.) / совокупный месячный доход;
* MonthlyIncome	– месячный доход в долларах;
* NumberOfDependents – число человек в семье кредитозаёмщика.

In [13]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

import matplotlib.pyplot as plt
import random
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [14]:
def impute_nan_with_median(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table

In [15]:
data = pd.read_csv('credit_scoring_sample.csv', sep=";")
data.head()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


In [16]:
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

['age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'NumberOfTimes90DaysLate',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'MonthlyIncome',
 'NumberOfDependents']

In [17]:
table = impute_nan_with_median(data)

In [18]:
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

In [19]:
y.value_counts()

0    35037
1    10026
Name: SeriousDlqin2yrs, dtype: int64

Задайте решающее дерево, пользуясь встроенной функцией `DecisionTreeClassifier` с параметрами `random_state=17` и `class_weight='balanced'`.

In [20]:
clf = DecisionTreeClassifier(random_state=17, class_weight='balanced')

Используйте функцию `GridSearchCV` для выбора оптимального набора гиперпараметров для указанной задачи. В качестве метрики качества возьмите ROC AUC.

In [21]:
max_depth_values = [3, 5, 6, 7, 9]
max_features_values = [4, 5, 6, 7]
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

Зафиксируйте кросс-валидацию с помощью функции `StratifiedKFold` на 5 разбиений с перемешиванием, `random_state=17`.

In [22]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [23]:
grid = GridSearchCV(clf, tree_params, cv=skf, scoring='roc_auc')

In [24]:
grid.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=17),
             param_grid={'max_depth': [3, 5, 6, 7, 9],
                         'max_features': [4, 5, 6, 7]},
             scoring='roc_auc')

Какое максимальное значение ROC AUC получилось?

In [25]:
grid.best_params_, grid.best_score_

({'max_depth': 7, 'max_features': 6}, 0.8189076588591803)

# Реализация случайного леса

Теперь реализуйте случайный лес. В качестве базового алгоритма здесь всегда выступает `DecisionTreeClassifier`.

На каждом шаге алгоритма необходимо запоминать индексы признаков, которые участвовали в обучении леса.

- В методе `fit` в цикле (`i` от 0 до `n_estimators-1`) фиксируйте seed, равный (`random_state + i`). Это нужно для того, чтобы на каждой итерации seed был новый, при этом все значения можно было бы воспроизвести.
- Зафиксировав seed, сделайте bootstrap-выборку (т.е. с замещением) из множества id объектов. Размер bootstrap-выборки = размеру исходной.
- Зафиксировав seed, выберите **без замещения** `max_features` признаков, сохраните список выбранных id признаков в `self.feat_ids_by_tree`.
- Обучите дерево с теми же `max_depth`, `max_features` и `random_state`, что и у `RandomForestClassifierCustom` на выборке с нужным подмножеством объектов и признаков.
- В методе `predict_proba` у тестовой выборки нужно взять те признаки, на которых соответствующее дерево обучалось, и сделать прогноз вероятностей (`predict_proba` уже для дерева, вернуть вероятности класса 1). Метод должен вернуть усреднение прогнозов по всем деревьям.

In [34]:
from sklearn.base import BaseEstimator

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=10, max_depth=5, max_features=4, random_state=5):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        # в данном списке будем хранить отдельные деревья
        self.trees = []
        # тут будем хранить списки индексов признаков, на которых обучалось каждое дерево 
        self.feat_ids_by_tree = []
        
    def fit(self, X, y):
        for j in range(self.n_estimators-1):

            features = random.choices(X.columns, k = self.max_features)
            self.feat_ids_by_tree.append(features)
    

            X1 = X.sample(n=int(0.66*len(X)), random_state=j)
            y1 = y.sample(n=int(0.66*len(X)), random_state=j)
            X_new = X1[features]
            
            clf = DecisionTreeClassifier(random_state=self.random_state, max_depth=self.max_depth, 
                    max_features=self.max_features).fit(X_new, y1)
            self.trees.append(clf)
            
        
    def predict_proba(self, X):
        res = []
        for (clf, features) in zip(self.trees, self.feat_ids_by_tree):
            res.append(clf.predict_proba(X[features])[:,1])
        temp = sum(np.array(res))/self.n_estimators
        return temp

Проведите кросс-валидацию. Какое получилось среднее значение ROC AUC на кросс-валидации? Сравните качество вашей реализации с реализацией `RandomForestClassifier` из `sklearn`. Аналогично предыдущему заданию, подберите гиперпараметры для случайного леса.

In [35]:
clf_rf = RandomForestClassifierCustom(n_estimators=15, max_depth=3, max_features=4, random_state=17)
clf_rf.fit(X,y)

In [36]:
res = clf_rf.predict_proba(X)
res

array([0.09006872, 0.11433497, 0.16205167, ..., 0.1991626 , 0.26591971,
       0.36885457])